In [ ]:


Please execute the following Python code: result = 2 + 2; print(f'The result is: {result}')



Okay, let's integrate a revolving proxy system into OpenManus. Since OpenManus uses Playwright for its browser interactions (and Playwright has built-in proxy support), we'll focus on integrating the proxy rotation there. We'll also add a simple proxy management system, drawing inspiration from your provided example, but adapting it for asynchronous operation and Playwright.
Here's a breakdown of the changes and additions, followed by the code modifications:
    
Proxy Manager (browser_use/browser/proxy.py):
Creates a new ProxyManager class.
fetch_proxies(): Fetches proxies from the ProxyScrape API (asynchronously, using aiohttp). This is a placeholder; in a real-world scenario, you'd likely want to:
Use a more robust proxy provider.
Cache proxies locally.
Implement more sophisticated proxy validation and health checks.
Potentially integrate with a proxy management service.
get_proxy(): Returns the next proxy from the cycle (using itertools.cycle).
update_proxies(): Refreshes the proxy list.
Uses async and await for non-blocking I/O.
Integrate with BrowserConfig (browser_use/browser/browser.py):
Modify BrowserConfig to accept an optional ProxyManager instance. This allows flexibility: you can use OpenManus without proxies, or with a single proxy, or with a rotating proxy setup.
Update the start() method of the Browser class:
If a ProxyManager is provided, get the next proxy from it before launching the browser.
Use the proxy settings in the launch_options.
Update config (config/config.toml and app/config.py) and app/tool/browser_use.py:
Update the config files to have a new use_proxy_revolver flag to define whether to use proxy rotation.
Load a ProxyManager object on the Browser object only when the flag is set.
Error Handling (Implicit in Playwright): Playwright has robust error handling. If a connection to a proxy fails, Playwright will raise an exception. The BrowserUseTool already catches PlaywrightError, so this is largely handled. A more sophisticated implementation might add retry logic within the ProxyManager to try a different proxy on specific connection errors.
Here are the code modifications:


In [ ]:

# browser_use/browser/proxy.py (NEW FILE)
import aiohttp
from itertools import cycle
from typing import List, Optional

from pydantic import BaseModel


class ProxySettings(BaseModel):
    server: str
    username: Optional[str] = None
    password: Optional[str] = None


class ProxyManager:
    def __init__(self, source_url: str = "https://api.proxyscrape.com/v2/?request=getproxies&protocol=http&timeout=10000&country=all&ssl=yes&anonymity=all"):
        self.proxies: List[ProxySettings] = []
        self.proxy_pool = None  # Will be an itertools.cycle object
        self.source_url = source_url

    async def fetch_proxies(self) -> List[ProxySettings]:
        """Fetch a list of proxies from ProxyScrape (or another source)."""
        async with aiohttp.ClientSession() as session:
            async with session.get(self.source_url) as response:
                if response.status == 200:
                    proxies_text = await response.text()
                    proxies_list = proxies_text.splitlines()
                    # Filter out empty strings and create ProxySettings
                    return [
                        ProxySettings(server=proxy)
                        for proxy in proxies_list if proxy
                    ]
        return []  # Return empty list on failure

    async def update_proxies(self):
        """Update the internal proxy list."""
        self.proxies = await self.fetch_proxies()
        if self.proxies:  # Only create the cycle if we have proxies
            self.proxy_pool = cycle(self.proxies)
        else:
            self.proxy_pool = None # Set to None if no proxies

    def get_proxy(self) -> Optional[ProxySettings]:
        """Return the next proxy in the cycle, or None if no proxies."""
        if self.proxy_pool:
             return next(self.proxy_pool)
        return None



In [ ]:


# browser_use/browser/browser.py (MODIFIED)
import asyncio
from typing import List, Optional, Union, Dict
from playwright.async_api import (
    async_playwright,
    Browser as PlaywrightBrowser,
    BrowserContext as PlaywrightBrowserContext,
    Page,
    Playwright,
)
from pydantic import BaseModel, Field
from browser_use.browser.context import (
    BrowserContextConfig,
    BrowserContextManager,
    BrowserContextWrapper,
)
from browser_use.browser.proxy import ProxyManager, ProxySettings # Import ProxyManager


class BrowserConfig(BaseModel):
    headless: bool = Field(False, description="Whether to run browser in headless mode")
    disable_security: bool = Field(
        True, description="Disable browser security features"
    )
    extra_chromium_args: List[str] = Field(
        default_factory=list, description="Extra arguments to pass to the browser"
    )
    chrome_instance_path: Optional[str] = Field(
        None, description="Path to a Chrome instance to use"
    )
    wss_url: Optional[str] = Field(
        None, description="Connect to a browser instance via WebSocket"
    )
    cdp_url: Optional[str] = Field(
        None, description="Connect to a browser instance via CDP"
    )
    proxy: Optional[ProxySettings] = Field(
        None, description="Proxy settings for the browser"
    )
    proxy_manager: Optional[ProxyManager] = Field(
        None, description="Proxy Manager for rotating proxies"
    )  # Add ProxyManager


class Browser:
    """
    Manages a Playwright browser instance, contexts, and pages.
    """

    def __init__(self, config: Optional[BrowserConfig] = None) -> None:
        self.config = config or BrowserConfig()
        self.playwright: Optional[Playwright] = None
        self.browser: Optional[PlaywrightBrowser] = None
        self.context_manager = BrowserContextManager()

    async def start(self) -> None:
        """Starts the Playwright browser instance."""
        if self.browser and self.browser.is_connected():
            return  # already started

        self.playwright = await async_playwright().start()

        launch_options = {}
        if self.config.extra_chromium_args:
            launch_options["args"] = self.config.extra_chromium_args
        if self.config.chrome_instance_path:
            launch_options["executable_path"] = self.config.chrome_instance_path
        if self.config.headless:
            launch_options["headless"] = self.config.headless

        # Use ProxyManager if available, otherwise use single proxy settings
        if self.config.proxy_manager:
            await self.config.proxy_manager.update_proxies()  # Fetch initial proxies
            proxy_settings = self.config.proxy_manager.get_proxy()
            if proxy_settings:
                launch_options["proxy"] = {
                    "server": proxy_settings.server,
                    "username": proxy_settings.username,
                    "password": proxy_settings.password,
                }
        elif self.config.proxy:
            launch_options["proxy"] = {
                "server": self.config.proxy.server,
                "username": self.config.proxy.username,
                "password": self.config.proxy.password,
            }

        if self.config.wss_url:
            self.browser = await self.playwright.chromium.connect_over_cdp(
                self.config.wss_url
            )
        elif self.config.cdp_url:
            self.browser = await self.playwright.chromium.connect_cdp(self.config.cdp_url)
        else:
            self.browser = await self.playwright.chromium.launch(**launch_options)

    async def new_context(
        self, context_id: str = "default", config: Optional[BrowserContextConfig] = None
    ) -> BrowserContextWrapper:
        """
        Creates a new browser context.
        """
        await self.start()  # Ensure browser is started
        if not config:
            config = BrowserContextConfig()
        if not self.browser:
            raise RuntimeError("Browser not started. Call start() first.")

        return await self.context_manager.create_context(self.browser, context_id, config)

    async def get_context(self, context_id: str) -> BrowserContextWrapper:
        """
        Retrieves an existing browser context.
        """
        return await self.context_manager.get_context(context_id)

    async def close_context(self, context_id: str) -> None:
        """Closes a specific browser context."""
        await self.context_manager.close_context(context_id)

    async def close(self) -> None:
        """Closes the browser and all associated contexts."""
        if self.browser:
            await self.context_manager.close_all_contexts()
            await self.browser.close()
            self.browser = None
        if self.playwright:
            await self.playwright.stop()
            self.playwright = None

    # Add a helper method to check if the browser is running
    def is_running(self) -> bool:
        """Checks if the browser instance is currently running."""
        return self.browser is not None and self.browser.is_connected()

    async def __aenter__(self):
        await self.start()
        return self

    async def __aexit__(self, exc_type, exc_val, exc_tb):
        await self.close()




In [ ]:


# app/config.py (MODIFIED)
import threading
import tomllib
from pathlib import Path
from typing import Dict, List, Optional

from pydantic import BaseModel, Field
from browser_use.browser.proxy import ProxyManager #import proxy manager


def get_project_root() -> Path:
    """Get the project root directory"""
    return Path(__file__).resolve().parent.parent


PROJECT_ROOT = get_project_root()
WORKSPACE_ROOT = PROJECT_ROOT / "workspace"


class LLMSettings(BaseModel):
    model: str = Field(..., description="Model name")
    base_url: str = Field(..., description="API base URL")
    api_key: str = Field(..., description="API key")
    max_tokens: int = Field(4096, description="Maximum number of tokens per request")
    temperature: float = Field(1.0, description="Sampling temperature")
    api_type: str = Field(..., description="AzureOpenai or Openai")
    api_version: str = Field(..., description="Azure Openai version if AzureOpenai")


class ProxySettings(BaseModel):
    server: str = Field(None, description="Proxy server address")
    username: Optional[str] = Field(None, description="Proxy username")
    password: Optional[str] = Field(None, description="Proxy password")


class BrowserSettings(BaseModel):
    headless: bool = Field(False, description="Whether to run browser in headless mode")
    disable_security: bool = Field(
        True, description="Disable browser security features"
    )
    extra_chromium_args: List[str] = Field(
        default_factory=list, description="Extra arguments to pass to the browser"
    )
    chrome_instance_path: Optional[str] = Field(
        None, description="Path to a Chrome instance to use"
    )
    wss_url: Optional[str] = Field(
        None, description="Connect to a browser instance via WebSocket"
    )
    cdp_url: Optional[str] = Field(
        None, description="Connect to a browser instance via CDP"
    )
    proxy: Optional[ProxySettings] = Field(
        None, description="Proxy settings for the browser"
    )
    new_context_config: Optional[Dict] = Field(
        None, description="Additional context configuration for the browser"
    )
    use_proxy_revolver: bool = Field(False, description="Wether to use proxy rotation") #add new flag
    proxy_manager: Optional[ProxyManager] = None


class AppConfig(BaseModel):
    llm: Dict[str, LLMSettings]
    browser_config: Optional[BrowserSettings] = Field(
        None, description="Browser configuration"
    )

    class Config:
        arbitrary_types_allowed = True


class Config:
    _instance = None
    _lock = threading.Lock()
    _initialized = False

    def __new__(cls):
        if cls._instance is None:
            with cls._lock:
                if cls._instance is None:
                    cls._instance = super().__new__(cls)
        return cls._instance

    def __init__(self):
        if not self._initialized:
            with self._lock:
                if not self._initialized:
                    self._config = None
                    self._load_initial_config()
                    self._initialized = True

    @staticmethod
    def _get_config_path() -> Path:
        root = PROJECT_ROOT
        config_path = root / "config" / "config.toml"
        if config_path.exists():
            return config_path
        example_path = root / "config" / "config.example.toml"
        if example_path.exists():
            return example_path
        raise FileNotFoundError("No configuration file found in config directory")

    def _load_config(self) -> dict:
        config_path = self._get_config_path()
        with config_path.open("rb") as f:
            return tomllib.load(f)

    def _load_initial_config(self):
        raw_config = self._load_config()
        base_llm = raw_config.get("llm", {})
        llm_overrides = {
            k: v for k, v in raw_config.get("llm", {}).items() if isinstance(v, dict)
        }

        default_settings = {
            "model": base_llm.get("model"),
            "base_url": base_llm.get("base_url"),
            "api_key": base_llm.get("api_key"),
            "max_tokens": base_llm.get("max_tokens", 4096),
            "temperature": base_llm.get("temperature", 1.0),
            "api_type": base_llm.get("api_type", ""),
            "api_version": base_llm.get("api_version", ""),
        }

        # handle browser config.
        browser_config = raw_config.get("browser", {})
        browser_settings = None

        if browser_config:
            # handle proxy settings.
            proxy_config = browser_config.get("proxy", {})
            proxy_settings = None

            if proxy_config and proxy_config.get("server"):
                proxy_settings = ProxySettings(
                    **{
                        k: v
                        for k, v in proxy_config.items()
                        if k in ["server", "username", "password"] and v
                    }
                )

            # filter valid browser config parameters.
            valid_browser_params = {
                k: v
                for k, v in browser_config.items()
                if k in BrowserSettings.__annotations__ and v is not None
            }

            # if there is proxy settings, add it to the parameters.
            if proxy_settings:
                valid_browser_params["proxy"] = proxy_settings
            
            #set proxy_manager to None
            valid_browser_params["proxy_manager"] = None
            # only create BrowserSettings when there are valid parameters.
            if valid_browser_params:
                browser_settings = BrowserSettings(**valid_browser_params)

        config_dict = {
            "llm": {
                "default": default_settings,
                **{
                    name: {**default_settings, **override_config}
                    for name, override_config in llm_overrides.items()
                },
            },
            "browser_config": browser_settings,
        }

        self._config = AppConfig(**config_dict)

    @property
    def llm(self) -> Dict[str, LLMSettings]:
        return self._config.llm

    @property
    def browser_config(self) -> Optional[BrowserSettings]:
        return self._config.browser_config

config = Config()




In [ ]:


# app/tool/browser_use.py (MODIFIED)
import asyncio
from typing import List, Optional, Union, Dict, Any

from pydantic import BaseModel, Field, ValidationError

from app.tool.base import BaseTool, ToolResult, AgentAwareTool
from app.config import config  # Correct import
from browser_use.browser.browser import Browser, BrowserConfig  # Correct import
from browser_use.browser.context import BrowserContext, BrowserContextConfig
from browser_use.dom.dom_service import DomService  # Correct import
from browser_use.browser.proxy import ProxyManager
from playwright.async_api import Error as PlaywrightError


# define actions
class AvailableActions(BaseModel):
    """
    Represents the available actions that can be performed with the browser tool.
    """

    actions: List[str] = Field(..., description="List of available actions")


class NavigateAction(BaseModel):
    """Navigates to a URL."""

    action: str = Field(
        "navigate_to", const=True, description="Action to navigate to a website"
    )
    url: str = Field(..., description="URL to navigate to")


class ClickAction(BaseModel):
    """Clicks on a element."""

    action: str = Field(
        "click_element", const=True, description="Action to click on a element"
    )
    selector: str = Field(..., description="The selector of the element to click")


class InputTextAction(BaseModel):
    """Inputs text into a field."""

    action: str = Field(
        "input_text", const=True, description="Action to input text into a field"
    )
    selector: str = Field(..., description="The selector of the element to input")
    text: str = Field(..., description="text to fill")


class GetLinksAction(BaseModel):
    action: str = Field(
        "get_links", const=True, description="Get all links in the current page"
    )


class ScrollPageAction(BaseModel):
    action: str = Field("scroll_page", const=True, description="Scroll the page")
    scroll_amount: int = Field(
        ...,
        description="The amount to scroll. Positive to scroll down, negative to scroll up.",
    )


class GetCurrentStateAction(BaseModel):
    action: str = Field(
        "get_current_state", const=True, description="Get the current state of the browser"
    )


class SwitchTabAction(BaseModel):
    action: str = Field("switch_to_tab", const=True, description="Switch to a tab")
    tab_index: int = Field(..., description="Index of the tab to switch")


class CloseTabAction(BaseModel):
    action: str = Field("close_tab", const=True, description="Close a tab")
    tab_index: int = Field(..., description="Index of the tab to close")


class CreateNewTabAction(BaseModel):
    action: str = Field(
        "create_new_tab",
        const=True,
        description="Create a new tab, optionally navigating to a URL.",
    )
    url: Optional[str] = Field(None, description="URL to navigate to in the new tab")


class RefreshAction(BaseModel):
    action: str = Field("refresh_page", const=True, description="Refresh current page")


# ---

# Union of all possible action types
Action = Union[
    NavigateAction,
    ClickAction,
    InputTextAction,
    GetLinksAction,
    ScrollPageAction,
    GetCurrentStateAction,
    SwitchTabAction,
    CloseTabAction,
    CreateNewTabAction,
    RefreshAction,
]


class BrowserUseTool(BaseTool, AgentAwareTool):
    name: str = "browser_use"
    description: str = "Used to browse the web"
    browser: Browser = Field(
        default_factory=lambda: Browser(config.browser_config)
    )  # Use default config
    browser_context: Optional[BrowserContext] = None
    parameters: Dict = {"type": "object", "properties": {}}

    def __init__(self, **data):
        super().__init__(**data)
        self.description = (
            f"Used to browse the web. Available actions: {self.get_available_actions()}"
        )
        # Initialize ProxyManager only if use_proxy_revolver is True
        if self.browser.config.use_proxy_revolver:
             self.browser.config.proxy_manager = ProxyManager()

    async def execute(self, **kwargs) -> ToolResult:
        try:
            action = self.parse_action(**kwargs)
            result = await self._execute_action(action)

        except ValidationError as e:
            return ToolResult(error=str(e))
        except PlaywrightError as e:
            return ToolResult(error=str(e))
        except ValueError as e:
            return ToolResult(error=str(e))
        except Exception as e:
            return ToolResult(error=str(e))
        return result

    def get_available_actions(self) -> str:
        """Returns a string listing all available actions."""
        available_actions_model = AvailableActions(
            actions=[
                NavigateAction(url="").action,
                ClickAction(selector="").action,
                InputTextAction(selector="", text="").action,
                GetLinksAction().action,
                ScrollPageAction(scroll_amount=0).action,
                GetCurrentStateAction().action,
                SwitchTabAction(tab_index=0).action,
                CloseTabAction(tab_index=0).action,
                CreateNewTabAction().action,
                RefreshAction().action,
            ]
        )
        return ", ".join(available_actions_model.actions)

    def parse_action(self, **kwargs) -> Action:
        """Parses the action based on kwargs."""
        action_name = kwargs.get("action")

        if action_name == "navigate_to":
            return NavigateAction(**kwargs)
        elif action_name == "click_element":
            return ClickAction(**kwargs)
        elif action_name == "input_text":
            return InputTextAction(**kwargs)
        elif action_name == "get_links":
            return GetLinksAction(**kwargs)
        elif action_name == "scroll_page":
            return ScrollPageAction(**kwargs)
        elif action_name == "get_current_state":
            return GetCurrentStateAction(**kwargs)
        elif action_name == "switch_to_tab":
            return SwitchTabAction(**kwargs)
        elif action_name == "close_tab":
            return CloseTabAction(**kwargs)
        elif action_name == "create_new_tab":
            return CreateNewTabAction(**kwargs)
        elif action_name == "refresh_page":
            return RefreshAction(**kwargs)
        else:
            raise ValueError(f"Unknown action: {action_name}")

    async def _ensure_browser_context(self) -> BrowserContext:
        """Ensures that the browser and a context are running"""
        # start browser
        if not self.browser or not self.browser.is_running():
            await self.browser.start()
        # get context
        if not self.browser_context:
            context_wrapper = await self.browser.new_context()  # Use default context
            self.browser_context = BrowserContext(context_wrapper)  # Create the context wrapper

        return self.browser_context

    async def _execute_action(self, action: Action) -> ToolResult:
        """Executes the given action using Playwright."""
        context = await self._ensure_browser_context()

        if isinstance(action, NavigateAction):
            await context.navigate_to(action.url)
            return ToolResult(output=f"Navigated to {action.url}")

        elif isinstance(action, ClickAction):
            await context.click_element(action.selector)
            return ToolResult(output=f"Clicked element: {action.selector}")

        elif isinstance(action, InputTextAction):
            await context.input_text(action.selector, action.text)
            return ToolResult(output=f"Input text into element: {action.selector}")

        elif isinstance(action, GetLinksAction):
            links = await context.get_links()
            return ToolResult(output=f"Links: {links}")

        elif isinstance(action, ScrollPageAction):
            await context.scroll_page(action.scroll_amount)
            return ToolResult(output=f"Scrolled page by {action.scroll_amount}")
        elif isinstance(action, GetCurrentStateAction):
            state = await context.get_all_tabs_info()
            return ToolResult(output=f"Current State {state}")

        elif isinstance(action, SwitchTabAction):
            await context.switch_to_tab(action.tab_index)
            return ToolResult(output=f"Switched to tab: {action.tab_index}")
        elif isinstance(action, CloseTabAction):
            await context.close_tab(action.tab_index)
            return ToolResult(output=f"Closed tab: {action.tab_index}")

        elif isinstance(action, CreateNewTabAction):
            await context.create_new_tab(action.url)
            return ToolResult(output=f"New tab opened")

        elif isinstance(action, RefreshAction):
            await context.refresh_page()
            return ToolResult(output="Page refreshed")

    async def cleanup(self) -> None:
        """Clean up resources (close browser)."""
        if self.browser:
            await self.browser.close()
            self.browser = None
            self.browser_context = None

    async def get_current_state(self) -> ToolResult:
        """Get the current state of the first context"""
        context = await self._ensure_browser_context()  # get default context
        state = await context.get_all_tabs_info()
        return ToolResult(output=str(state))





In [ ]:


# config/config.toml
[llm.default]
model = "gpt-3.5-turbo-1106"
base_url = "https://api.openai.com/v1"
api_key = "YOUR_OPENAI_API_KEY"
api_type = "openai"
api_version = ""

[browser]
headless = False
use_proxy_revolver = True #Add this line




In [ ]:

Key changes and why:
browser_use/browser/proxy.py: This new file contains the ProxyManager class, responsible for fetching and cycling through proxies. It uses aiohttp for asynchronous HTTP requests.
browser_use/browser/browser.py:
The BrowserConfig now has an optional proxy_manager field.
The start() method checks for proxy_manager. If present, it gets a proxy before launching the browser.
config/config.toml: A use_proxy_revolver setting is added to enable/disable.
app/config.py:
Updated to include the use_proxy_revolver flag.
The proxy_manager attribute in BrowserSettings is initialized to None.
app/tool/browser_use.py:
The BrowserUseTool's __init__ method now conditionally creates a ProxyManager instance only if self.browser.config.use_proxy_revolver is true. This is critical: it connects the configuration setting to the actual use of the proxy manager.
Updated create.sh:
